Week 5, Assignment C1:

Logistic Regression with Yahoo! Finance API in Python

In this notebook, we will study Logistic Regression using the Yahoo! Finance API in Python.

Please complete the lines bellow where you see "#TODO".

In [1]:
#Objective 1.
#Use this cell to import the Numpy (as np), Pandas (as pd), and YFinance (as yf) packages.

import numpy as np
import pandas as pd
import yfinance as yf

#Our work will also require some components of the Sklearn and Pandas_Datareader packages as imported below:

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from pandas_datareader import data as pdr
yf.pdr_override()

In [2]:
#Objective 2.
#Select a stock symbol for a stock whose historical data is available on the Yahoo! Finance website.  Store the
#string of your chosen symbol to the new variable "stock_symbol".

stock_symbol = 'AAPL'

In [3]:
#Objective 3.
#Use the function "pdr.get_data_yahoo(stock_symbol, start_date, end_date)" to generate a Pandas dataframe of
#historical stock data for your chosen stock.  Retain only the first four columns of the dataframe.  Be sure to 
#drop any rows containing NaN's, and take a peek at the resulting dataframe to make sure everything looks good.

df = pdr.get_data_yahoo(stock_symbol, '2000-11-08', '2021-11-08' )
#TODO
df = df.iloc[:, :4]
#TODO
df = df.dropna()
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close
Date,,,,
2000-11-07,0.383929,0.389509,0.371652,0.380580
2000-11-08,0.381696,0.382813,0.353795,0.358259
2000-11-09,0.354911,0.366071,0.340402,0.360491
2000-11-10,0.345703,0.354911,0.340402,0.340402
2000-11-13,0.334821,0.357143,0.325893,0.345982
...,...,...,...,...
2021-11-01,148.990005,149.699997,147.800003,148.960007
2021-11-02,148.660004,151.570007,148.649994,150.020004
2021-11-03,150.389999,151.970001,149.820007,151.490005


In [4]:
#Objective 4.
#Use the historical data to define some predictor variables.  Add these variables to the dataframe.  Include, at a
#minimum, (Predictor.I) the rolling average closing price over the past fifteen (15) days and (Predictor.II) the
#change in opening price over the past one (1) day.  Be sure to again drop any rows containing NaN's, and take a
#peek at the resulting dataframe to make sure everything looks good.  Store the dataframe as the new variable "X".

df['Close_15_Rolling'] = df['Close'].rolling(window=15).mean()
#TODO
df['Open_1_Change'] = df['Open']-df['Open'].shift(1)

#TODO
df = df.dropna()
#TODO
X = df
#TODO
X

,Open,High,Low,Close,Close_15_Rolling,Open_1_Change
Date,,,,,,
2000-11-28,0.333705,0.339286,0.320313,0.321987,0.345127,-0.021206
2000-11-29,0.323103,0.327009,0.308036,0.313616,0.340662,-0.010602
2000-11-30,0.297991,0.303571,0.287946,0.294643,0.336421,-0.025112
2000-12-01,0.303571,0.312500,0.300223,0.304688,0.332701,0.005580
2000-12-04,0.306920,0.306920,0.293527,0.297991,0.329874,0.003349
...,...,...,...,...,...,...
2021-11-01,148.990005,149.699997,147.800003,148.960007,147.460001,1.770004
2021-11-02,148.660004,151.570007,148.649994,150.020004,148.027335,-0.330002
2021-11-03,150.389999,151.970001,149.820007,151.490005,148.732668,1.729996


In [5]:
#Objective 5.
#Define the target or dependent variable to be one (1) if the change in closing price over the past one (1) day is
#nonnegative and negative one (-1) if the change in closing price over the past one (1) day is negative.  This
#variable's values should be forward-looking (i.e., you should subtract today's price from tomorrow's price rather
#than subtracting yesterday's price from today's price).  Store the resulting values as the new variable "y".  You
#may find the "np.where(*args)" function to be useful.  Look it up in Numpy documentation for support.

y = np.where(df['Close'].shift(-1) - df['Close'] < 0, -1, 1)
y
#TODO

array([-1, -1,  1, ..., -1,  1,  1])

In [6]:
#Objective 6.
#Split the data into training and test sets, putting the first seventy percent (70%) of the data in the training
#set.

index = int(0.7*len(X))
X_train, X_test, y_train, y_test = X[:index],X[index:],y[:index],y[index:]


In [7]:
X_train

,Open,High,Low,Close,Close_15_Rolling,Open_1_Change
Date,,,,,,
2000-11-28,0.333705,0.339286,0.320313,0.321987,0.345127,-0.021206
2000-11-29,0.323103,0.327009,0.308036,0.313616,0.340662,-0.010602
2000-11-30,0.297991,0.303571,0.287946,0.294643,0.336421,-0.025112
2000-12-01,0.303571,0.312500,0.300223,0.304688,0.332701,0.005580
2000-12-04,0.306920,0.306920,0.293527,0.297991,0.329874,0.003349
...,...,...,...,...,...,...
2015-07-23,31.549999,31.772499,31.264999,31.290001,31.557833,1.052500
2015-07-24,31.330000,31.434999,30.975000,31.125000,31.525500,-0.219999
2015-07-27,30.772499,30.902500,30.530001,30.692499,31.471667,-0.557501


In [8]:
X_test

,Open,High,Low,Close,Close_15_Rolling,Open_1_Change
Date,,,,,,
2015-07-30,30.580000,30.642500,30.427500,30.592501,31.478500,-0.207500
2015-07-31,30.650000,30.660000,30.227501,30.325001,31.445500,0.070000
2015-08-03,30.375000,30.642500,29.379999,29.610001,31.325167,-0.275000
2015-08-04,29.355000,29.424999,28.312500,28.660000,31.142333,-1.020000
2015-08-05,28.237499,29.360001,28.025000,28.850000,30.952000,-1.117500
...,...,...,...,...,...,...
2021-11-01,148.990005,149.699997,147.800003,148.960007,147.460001,1.770004
2021-11-02,148.660004,151.570007,148.649994,150.020004,148.027335,-0.330002
2021-11-03,150.389999,151.970001,149.820007,151.490005,148.732668,1.729996


In [9]:
y_train

array([-1, -1,  1, ...,  1, -1, -1])

In [10]:
y_test

array([-1, -1, -1, ..., -1,  1,  1])

In [11]:
#Objective 7.
#Instantiate the Logistic Regression model object, and use its ".fit(*args)" method to fit the model to the 
#training data.

logistic = LogisticRegression()
logistic = logistic.fit(X_train, y_train)

In [12]:
#Objective 8.
#Examine the model's coefficients by using its ".coef_" method.

pd.DataFrame(zip(X.columns, np.transpose(logistic.coef_)))

,0,1
0,Open,[0.04995357688290602]
1,High,[0.13096859365622843]
2,Low,[0.4527115411537033]
3,Close,[-0.533940586599525]
4,Close_15_Rolling,[-0.10025977164235089]
5,Open_1_Change,[-0.07798385218670081]


In [13]:
#Objective 9.
#Use the model's ".predict_proba(*args)" and ".predict(*args)" methods to generate predictions over the test set.

probabilities =logistic.predict_proba(X_test)
predictions = logistic.predict(X_test)


In [14]:
#Objective 10.
#Use the function "metrics.confusion_matrix(*args)" to create a confusion matrix comparing the predicted and true
#classification labels over the test set.
metrics.confusion_matrix(y_test,predictions)
#TODO

array([[363, 371],
       [369, 478]])

In [15]:
#Objective 11.
#Calculate the model's accuracy on the test set using its ".score(*args)" method.
logistic.score(X_test,y_test)
#TODO

0.5319418089816572

In [16]:
#Objective 12.
#Use five-fold cross validation to cross-check the accuracy of the model over different held-out test sets.  This
#is where you should use the function "cross_val_score(*args)".

cross_val = cross_val_score(logistic,X,y,cv=5)

#cross_val = cross_val_score(logistic, X, y, scoring='accuracy', cv = 5)
cross_val

array([0.52751423, 0.52751423, 0.53510436, 0.53320683, 0.52421652])